In [2]:
# This document/.ipynb file was originaly created by Sacha Dubrulle in Python 2 
# and further worked upon and converted to Python 3 by Thomas Valcke.
# This notebook has been converted to Python 3 because the syntax changed with Python 3.

# This notebook handels the data preperation and is the largest of the files.
# 5 Problems remain:

# 1. # Mine stock prices -> this gets its data from Yahoo.finance, This has been discontinued so can not work.

# 2. 


# <span style = 'color:darkred'> Imports </span style>

In [1]:
################################ NEW IMPORT ###############################
import os
import sys
import glob
#os.environ['SPARK_HOME'] = 'D:/School/STAGE_BP/spark'
################################ JUPYTER ###################################
from IPython.core.display import display, HTML
from IPython.display import Image
import imp
display(HTML("<style>.container { width:100% !important; }</style>"))

################################ SPARK ###################################


import pyspark

from pyspark.sql import SQLContext, SparkSession

try:
    sc = pyspark.SparkContext(appName = 'App')
    spark = SparkSession(sparkContext=sc)
    print("SparkSession initialized")
except ValueError:
    print("SparkSession already initialized")
    
    
from pyspark.sql.functions import udf, explode, lower, regexp_extract, split, col, regexp_replace,upper
from pyspark.sql.types import StringType, TimestampType, IntegerType, ArrayType, DoubleType, BooleanType
from pyspark.ml.feature import Binarizer, StringIndexer, StringIndexerModel, OneHotEncoder, MinMaxScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

################################ PYTHON ###################################
    
from datetime import datetime
import re
import pandas as pd
pd.set_option("display.max_colwidth", -1)

import numpy as np
import urllib
get_ipython().run_line_magic('matplotlib', 'inline')
from matplotlib import pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, WeekdayLocator, DateFormatter
from dateutil.relativedelta import relativedelta

################################ Alpha Vantage API ###################################
import requests
import alpha_vantage
import csv
#This is our current api key, you might have to request a new one
AV_API_key="OES7ORJTQ21CYZM9"
AV_API_URL = "https://www.alphavantage.co/query" 
from datetime import datetime
date_start = datetime.now()

SparkSession initialized


In [31]:
spark=pyspark.sql.SparkSession.builder.appName('Bottom-Up-Analysis').config('spark.driver.memory','12G').getOrCreate()

In [3]:
#We initialize some colors for the graphs
colors = {"orange": np.array([255,128,14], dtype = np.float16),
          "blue": np.array([0,107,164], dtype = np.float16),
          "green": np.array([44, 160, 44], dtype = np.float16)
         }

for k in list(colors.keys()):
    colors[k] = np.divide(colors[k], 255)

This notebook is based on data that was mined from twitter. Primarily, twitter was mined using a list of keywords, all related to Initial Public Offerings, herinafter IPO's. The list of keywords contained the names and abbreviations or tickers for a number of companies that were know to perform their IPO soon, but also using some more generic words related to IPO's. The data ranges from the end of March 2016 until the end of march 2017 and hence comprises an entire year.

# <span style = 'color:darkred'> Import functions & make udfs </span style>

In [4]:
cd /home/mma/Downloads/IPO_project/commented_files/

/home/mma/Downloads/IPO_project/commented_files


In [5]:
# Read in functions from external file written in python and should be present in the same directory,
# Where this file is running from
import functions as f_ipo

f_ipo = imp.reload(f_ipo)

Comment Below Block

In [6]:
# Transform paterns. We are defining functions, with the functions we just imported.

#Extracting all cashtags from the tweets
udf_cashtag_re_extract = udf(f_ipo.cashtag_re_extract, ArrayType(elementType = StringType()))
#Gives the count of the mentions of the company's name
udf_contains_company_name = udf(f_ipo.contains_company_name, IntegerType())
non_empty_column_udf = udf(f_ipo.non_empty_column)
udf_contains_pattern = udf(f_ipo.contains_pattern)

# Define UDF to transform timestamp from StringType to TimestampType
udf_datatime = udf(lambda string: datetime.strptime(string, "%a %b %d %H:%M:%S +0000 %Y"), TimestampType())

udf_day_of_week = udf(lambda x: str(x.strftime('%A')), StringType())

In [6]:
import shutil # Package to move files by python commands
json_dir_name = '/home/mma/Downloads/IPO_project/tweets/' #file path
json_pattern = os.path.join(json_dir_name,'*.json') #searching for the JSON files 
file_list = glob.glob(json_pattern) # GLob for Unix path finding
#file_list= file_list[:100]

np_array_values=[] # Inititating the array
for files in file_list: #Loop to read all the files in the particular directory
    #print(files)
 #As we have different versions of JSON files from Twitter. Here We found some corrupted files, 
#then we moved those files into another folder .Therefore we used "try","except" 
    try:
        tweets=spark.read.json(files)
        if len(tweets.columns) > 27: # while merging all the files must contain equal number variables
            np_array_values.append(tweets)
        else:
            print("not enough columns for file {}".format(files))
            #Here we are moving all the files those contain less than 28 Variables
            shutil.move(files, "/home/mma/Downloads/IPO_project/tweets/corrupt_files/{}".format(files[-19:]))
    except:  # Skips the step in any case if the file contains error in the folder
        print("error with {}".format(files))
        shutil.move(files, "/home/mma/Downloads/IPO_project/tweets/corrupt_files/{}".format(files[-19:]))

In [ ]:
cd /home/S

# <span style = 'color:darkred'> Read in and basic transformations </span style>

In [7]:
#cashtag_expression is a regex that filters out everything that starts with $ and has 1-6 letters
cashtag_expression = "(\$[a-z]{1,6})+"
cashtag_expression_including_spaces = '\s\$[a-z]{1,6}\s' #add spaces to the cashtag
ipo_expression = ' #ipo | ipo ' 

#reads all companies in the list we've provided
with open("/home/mma/Downloads/IPO_project/company_names.txt", "r") as f:
    companies_list = f.readlines() 
    companies_list = [x.replace("\n", "").lower() for x in companies_list]

    
# Here I used the provided IPO tweets, 
# i am not 100% this is the data set used by the creator of this project but it runs and it is the closest i got.
# However if this data is not the same as used by the creator this would explain a lot of the random errors...
# original directory on the creators Laptop: /home/sachadubrulle/Data/tweets_IPO/done_batches/M*B3/*.json
# directory on the retailp server : /mnt/bdata1/twitter/archief/ipo/tweets/*.json
# other copy of IPO files :         /mnt/bdata4/ipo/tweets/*.json
df = (spark.read.json('/home/mma/Downloads/IPO_project/tweets/*.json')

      
        # Add lower text column and timestamp
        .select("*", 
               udf_datatime(col("created_at")).alias("timestamp"),
               lower(col("text")).alias("text_lower")
               )
        # Drop created_at string column
        .drop("created_at")
        # Add columns related to ipo, cashtags & retweet patterns
        .select("*", 
               udf_cashtag_re_extract('text_lower').alias("cashtags"),
               non_empty_column_udf(regexp_extract('text_lower', ipo_expression, 0)).alias("contains_ipo"),
               non_empty_column_udf(regexp_extract('text_lower', cashtag_expression, 0)).alias("contains_cashtag"),
               non_empty_column_udf(regexp_extract('text_lower', cashtag_expression_including_spaces, 0)).alias("contains_cashtag_with_spaces"),
               udf_contains_company_name("text_lower").alias("contains_company_name"),
               regexp_extract(str = "text", pattern = "^RT @[A-Za-z0-9]+: ", idx = 0).alias("retweet_pattern")
               )
        .select("*", 
               udf_contains_pattern("retweet_pattern").alias("contains_RT_pattern")
              )
     )
df = (df.withColumnRenamed("id_str", "tweet_id_str") #rename to avoid confusion with user_id, retweet_id, ...
        .withColumn("user_id_str", df.user.id_str) 
        .select("*")
     )

#df.cache()

#display(df['_corrupt_record'])
nbr_total = df.count()
print("The filtered dataset contains {} tweets".format(nbr_total))
print("\nThe columns in this dataset are the following:\n{}".format(str(df.columns)))

The filtered dataset contains 1610431 tweets

The columns in this dataset are the following:
['_corrupt_record', 'contributors', 'coordinates', 'display_text_range', 'entities', 'extended_entities', 'extended_tweet', 'favorite_count', 'favorited', 'filter_level', 'geo', 'id', 'tweet_id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place', 'possibly_sensitive', 'quote_count', 'quoted_status', 'quoted_status_id', 'quoted_status_id_str', 'reply_count', 'retweet_count', 'retweeted', 'retweeted_status', 'source', 'text', 'timestamp_ms', 'truncated', 'user', 'withheld_copyright', 'withheld_in_countries', 'withheld_scope', 'timestamp', 'text_lower', 'cashtags', 'contains_ipo', 'contains_cashtag', 'contains_cashtag_with_spaces', 'contains_company_name', 'retweet_pattern', 'contains_RT_pattern', 'user_id_str']


In [10]:
# check about below columns?????
#df.select("contains_ipo","contains_cashtag","contains_cashtag_with_spaces").filter("contains_cashtag > 0").show(20)

In [8]:
tickers=pd.read_excel('/home/mma/Downloads/IPO_project/IPO_Tickers_List/IPO_data_final_2.xlsx')
tickers=tickers.append(pd.read_excel('/home/mma/Downloads/IPO_project/IPO_Tickers_List/IPO_data_final.xlsx'))
tickers=tickers.append(pd.read_excel('/home/mma/Downloads/IPO_project/IPO_Tickers_List/IPO_data_final_3.xlsx'))

/home/mma//lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [9]:
tickers

,Exchange,IPODate,IPOPrice,LockupExpiration,Name,Name,QuietPeriodExpiration,Ticker,URL
0,NASDAQ,Priced (5/6/2016),$12.00,11/2/2016,"SPRING BANK PHARMACEUTICALS, INC.",NaN,6/15/2016,SBPH,www.nasdaq.com/markets/ipos/company/spring-bank-pharmaceuticals-inc-898133-80118
1,NASDAQ,Priced (4/7/2016),$10.00,10/4/2016,"AEGLEA BIOTHERAPEUTICS, INC.",NaN,5/17/2016,AGLE,www.nasdaq.com/markets/ipos/company/aeglea-biotherapeutics-inc-961076-78705
2,NYSE,Priced (4/21/2016),$22.00,10/18/2016,"AMERICAN RENAL ASSOCIATES HOLDINGS, INC.",NaN,5/31/2016,ARA,www.nasdaq.com/markets/ipos/company/american-renal-associates-holdings-inc-835298-79295
3,NYSE,Priced (10/6/2016),$18.00,4/4/2017,AQUAVENTURE HOLDINGS LTD,NaN,11/15/2016,WAAS,www.nasdaq.com/markets/ipos/company/aquaventure-holdings-ltd-766355-79466
4,NYSE,Priced (3/17/2017),$14.00,9/13/2017,PROPETRO HOLDING CORP.,NaN,4/26/2017,PUMP,www.nasdaq.com/markets/ipos/company/propetro-holding-corp-999784-82763
5,NASDAQ,Priced (7/20/2016),$15.00,1/16/2017,"AUDENTES THERAPEUTICS, INC.",NaN,8/29/2016,BOLD,www.nasdaq.com/markets/ipos/company/audentes-therapeutics-inc-953394-80103
6,NYSE,Priced (10/12/2016),$18.00,4/10/2017,AZURE POWER GLOBAL LTD,NaN,11/21/2016,AZRE,www.nasdaq.com/markets/ipos/company/azure-power-global-ltd-981770-80005
7,NASDAQ,Priced (3/23/2016),$15.00,9/19/2016,"CORVUS PHARMACEUTICALS, INC.",NaN,5/2/2016,CRVS,www.nasdaq.com/markets/ipos/company/corvus-pharmaceuticals-inc-951574-80105
8,NASDAQ,Priced (2/3/2016),$16.00,8/1/2016,"EDITAS MEDICINE, INC.",NaN,3/14/2016,EDIT,www.nasdaq.com/markets/ipos/company/editas-medicine-inc-973356-80107
9,NYSE,Filed (9/24/2014),"<span class=""nodata""> -- </span>","<span class=""nodata""> -- </span>",EXMAR ENERGY PARTNERS LP,NaN,"<span class=""nodata""> -- </span>",XMLP,www.nasdaq.com/markets/ipos/company/exmar-energy-partners-lp-945504-76551


In [10]:
tickers = tickers[tickers['LockupExpiration'].notnull()]

In [11]:
ticker_list=tickers["Ticker"].to_list()# Subsetting tickers variable and converting into list

In [12]:
print(ticker_list[1])

AGLE


In [13]:
len(ticker_list)

536

In [14]:
ticker_list = set(ticker_list) # removing duplicates from the tickers list£

In [15]:
len(ticker_list)

521

In [16]:
ticker_list=list(filter(lambda x : str(x) != 'nan' , list(ticker_list)))

In [17]:
len(ticker_list)

521

In [18]:
# REGEX to extract tickers or presence of IPO
ticker_expression_no_numbers = '(\s\$[A-Za-z]{1,6})+'
ticker_expression_including_numbers = '\$[A-Za-z0-9]{1,6}'
ipo_expression = '#ipo|ipo|#IPO|IPO'

df_tickers = (df.withColumn("tickers", regexp_extract('text', ticker_expression_no_numbers, 0))
                .withColumn("contains_ipo", regexp_extract('text', ipo_expression, 0))
            )

In [19]:
# Filter full dataset for tweets that contain either a ticker-pattern or the IPO pattern
df_tickers_and_ipo = df_tickers.filter((df_tickers["tickers"] != "") | (df_tickers["contains_ipo"] != ""))

print("There are {} tweets in the total set that containt the word ipo or a pattern that matches a ticker, including the dollar sign".format(df_tickers.count()))

#df_tickers_and_ipo.select("tweet_id_str", "contains_ipo", "tickers").show(5)
#df.createOrReplaceTempView("dfSQL")

There are 1610431 tweets in the total set that containt the word ipo or a pattern that matches a ticker, including the dollar sign


In [20]:
# Filter out the tweets that contain one or multiple tickers
df_tickers = (df_tickers_and_ipo.filter(df_tickers_and_ipo["tickers"] != ""))


df_tickers = (df_tickers.select('contributors','coordinates','display_text_range','entities','extended_entities','extended_tweet','favorite_count','favorited','filter_level','geo','id','tweet_id_str','in_reply_to_screen_name','in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place', 'possibly_sensitive', 'quote_count', 'quoted_status', 'quoted_status_id', 'quoted_status_id_str', 'reply_count', 'retweet_count', 'retweeted', 'retweeted_status', 'source', 'text', 'timestamp_ms', 'truncated', 'user', 'withheld_in_countries', 'timestamp', 'text_lower','cashtags','contains_ipo','contains_cashtag','contains_cashtag_with_spaces','contains_company_name','retweet_pattern','contains_RT_pattern','user_id_str', split(df_tickers.tickers, '\s\$').alias('tickers')))


df_tickers = df_tickers.withColumn("ticker", explode(df_tickers.tickers))
df_tickers = df_tickers.filter(df_tickers.ticker != "")

aggregated_tickers = (df_tickers.withColumn("ticker", upper(df_tickers.ticker))
                                .groupBy("ticker").count()
                                .withColumnRenamed('count', "ticker_occurence")
                     )
aggregated_tickers = aggregated_tickers.sort(aggregated_tickers.ticker_occurence.desc())

#aggregated_tickers.coalesce(1).write.csv("ticker_occurence_2016", mode = "overwrite")


In [21]:
aggregated_tickers.rdd.take(10)

[Row(ticker='BTC', ticker_occurence=2380),
 Row(ticker='SPY', ticker_occurence=1757),
 Row(ticker='ETH', ticker_occurence=1342),
 Row(ticker='AAPL', ticker_occurence=1293),
 Row(ticker='TSLA', ticker_occurence=1015),
 Row(ticker='AMZN', ticker_occurence=809),
 Row(ticker='FB', ticker_occurence=732),
 Row(ticker='AAGC', ticker_occurence=689),
 Row(ticker='PVTL', ticker_occurence=638),
 Row(ticker='NFLX', ticker_occurence=603)]

In [22]:
df_tickers=df_tickers.select('*').filter('lang == "en"')

In [23]:
x=df_tickers.toPandas()

In [24]:
cd

/home/mma


In [25]:
# loop to generate Individual JSON files per Ticker
import json
df_tickers.createOrReplaceTempView("tic")
for i in ticker_list:
    #x = df_tickers.filter('ticker == {}'.format(i))
    x = spark.sql("SELECT * from tic where ticker= \"{}\"".format(i)) # Here it selects only required Ticker file as per requirement
    print(x.count())
    print(i)
    try:
        x = x.toPandas()
        x.to_json("{}.json".format(i))
        #x.coalesce(1).write.format('json').save('tweets_{}.json'.format(i))
        '''with open('tweets_{}.json'.format(i), 'w') as json_file:  
            json.dump(x, json_file)'''
    except:
        print("Error")

5
TRTX
5
TRMT
0
TBRGU
0
TC
0
RTLR
2
CHRA
7
RYB
0
VAPO
13
LX
7
OVID
12
SPRO
1
THOR
0
ACAMU
8
URGN
4
XERS
5
CNST
0
ETTX
47
SE
0
PSTX
4
NCSM
12
CASA
0
MGP
13
BCML
1
JG
9
GDI
1
DAVA
0
HCCHU
7
OPTN
12
SECO
0
LEVB
10
QES
2
FNKO
29
ARMO
8
IFRX
0
PLAN
4
BJS
16
SEND
3
CBTX
1
MESA
0
SXTC
3
DOGZ
47
SMAR
0
ZOOX
4
CPUL
0
PCIM
17
EVLO
9
ZLAB
30
WOW
10
KALA
6
DESP
23
SFIX
0
ENVN
69
VKTX
6
AQUA
5
CRNX
10
EOLS
0
MSC
1
PLSE
0
EQ
78
FND
3
VRCA
5
RRR
6
VCTR
0
VPRL
16
EDIT
8
BPMP
14
STXB
1
GMR
15
ADIL
6
MGTA
0
TWAV
0
FTCH
37
ZS
9
APTX
0
HZ
1
FOCS
0
CTK
3
ESQ
5
AMGP
0
YZCM
0
MQAI
1
BRPAU
25
LEVL
0
XMLP
2
KRYS
0
WALL
1
AQST
2
CNACU
0
YMAB
0
NIU
2
SOI
8
YOGA
1
RUBY
0
VIOT
66
DBX
2
OSPRU
0
TPGHU
33
SOGO
10
RYTM
4
EAGLU
0
AFGL
3
CSSE
0
ESTA
0
ESTC
0
TAI
2
POLY
17
ATUS
10
MGTX
4
ARA
16
TNTR
8
GTHX
6
HAIR
0
PUYI
1
OPRA
0
TKKSU
6
BDFC
2
GWRS
16
SCWX
6
AVRO
4
CADE
0
USOD
0
FPACU
0
PSIT
6
TOCA
10
BXG
38
UBX
6
ALRN
0
SIBN
0
HOTH
0
CTRA
6
SBT
10
EVOP
0
BWAY
29
BHVN
0
LTN'U
2
CAAP
12
BOLD
3
UXIN
0
MDJH
1
PSAV
50
DOCU
3

In [26]:
ls

''$'\033'/                           FSCT.json          RBB.json
 AAAU.json                           FTACU.json         RCUS.json
 ABLX.json                           FTCH.json          RDFN.json
 ACAMU.json                          FTSI.json          REDU.json
 ACMR.json                           FTSV.json          REPL.json
 ADIL.json                           GBSG.json          REPX.json
 ADIN.json                           GCAN.json          RETA.json
 ADOM.json                           GDI.json           RETO.json
 ADT.json                            GEMP.json          RLTY.json
 ADV.json                            GH.json            RMED.json
 AFGL.json                           GLDM.json         "RMG'U.json"
 AGLE.json                           GMHIU.json         RNGR.json
 AGMH.json                           GMR.json           ROAD.json
 AGS.json                            GNPX.json          ROKU.json
 AKCA.json                           GNTY.json          RRI.json
 ALACU.jso

### <span style = 'color:darkgreen'> Filter out all available tickers and mine their stock prices  </span style>

In [30]:
pd.read_json("SE.json").text

0     #earnings scheduled after the market closes today:  $CRM $BZUN $HPQ $GME $HPE $GES $CPRT $CAL $SE $QADA… https://t.co/GkeI1ezmOj                
1     RT @TimDohrmann: #Sensera $SE1 CEO Ralph Schmitt presenting at Thursday’s #ASX #smallcap conference in #Dubai: 👀 the story with Ralph’s sli…    
10    Huge coup for #IoT #sensor developer #Sensera $SE1 securing Ralph Schmitt as CEO, ex-@ToshibaUSA &amp; @CypressSemi… https://t.co/KJy0y1JztP    
11    📈 in #driverless trucks = 📈 in demand for location tracking/collision avoidance #sensors. Positive for #Sensera $SE1 https://t.co/iNCwc3dqW8    
12    #IoT #microsensor biz #Sensera $SE1 achieves Sept qtr cash receipts +151%, points to great growth prospects:… https://t.co/B7ZCRtXeCW           
13    🎧 Listen in: #Sensera's Matt Morgan explains revenue impact for $SE1 of exclusive new chip supply deal: audio 📼 her… https://t.co/Zr2u3t9iQj    
14    30 mins til Sensera $SE1's Matt Morgan launches investor conf call re sales traction wit

In [ ]:
'''

# change treshold depending on the cashtag count. I used a small data set to test the code so i lowerd it to 2
# but once u start using bigger amounts of data u can up this to 500 or more depending on the results.
treshold = 30 # collecting IPO's that contains threshold amount of tweets.

#cashtags will now contain all the cashtags in the tweets, this includes duplicates.
cashtags = df.select(explode("cashtags").alias("cashtag"))
#cashtag_count will now contain all the cashtags with a count of how many times they occur.
cashtag_count = cashtags.groupBy("cashtag").count().sort("count", ascending = False)
#cashtag_count_above_treshold will only contain the cashtags above the treshold.
cashtag_count_above_treshold = cashtag_count.filter("count >= {}".format(treshold))

print(("Considering all tweets, there is a total amount of {} cashtags.\n" +  
       "In total, there are {} distinct cashtags.\n" +
       "{} of these cashtags occur more than {} times.").format(cashtags.count(), 
                                                                cashtag_count.distinct().count(), 
                                                                cashtag_count_above_treshold.count(),
                                                                treshold))

cashtag_count.show(5) # printing cashtags

#ticker_rdd will now only contain the cashtags above the treshold, these will be put into
#the ticker_list
ticker_rdd = cashtag_count_above_treshold.select("cashtag").collect()

ticker_list = []
for i in tickers:
    ticker_list.append(i.cashtag)
print(ticker_list[0:10])'''

Remove the current proffit file, because we are appending, not writing

In [28]:
rm /home/mma/Downloads/IPO_Mining/stock_mining/profit.csv

In [29]:
# Mine stock prices adjustedspark.sql("SELECT entities, ex
import time
problem_stocks = []
i = 1
writingerror=[]
readingerror=[]
#We'll iterate over all cashtags above the treshold, referd to as tickers.
for ticker in ticker_list:
    #We can only use the api 5 times per minute, that's why we sleep for a minute (+ some cushioning)
    #every 5 times
    if (i%5 == 0):
        time.sleep(65)
    try:
        #increase our counter, so all the csv files get the correct input
        i = i+1
        #fetching the stock prices, all info can be found here: https://www.alphavantage.co/documentation/
        text = urllib.request.urlopen("https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={}&outputsize=full&interval=60min&apikey={}&datatype=csv".format(ticker,AV_API_key)).read()
    except Exception as error:
        #If there is an error, the tickers get added to a list.
        print(error)
        print("Problem fetching {}".format(ticker))
        problem_stocks.append(ticker)
    print(ticker)
    #We split the text into an array, first we need to extract every line, these end in \\r\\n
    #then we split them on every comma, this is so we can convert them to a csv file later.
    textrows = str(text).split("\\r\\n")
    #textcsv = map(lambda x:(x.split(',')), textrows)
    try:
        values = str(textrows[-2]).split(",")        
        profit = round((float(values[-2]) - float(values[1]))/float(values[1]),4)
        x=[[profit,ticker]]
        print(x)
        try:
            with open("/home/mma/Downloads/IPO_project/stock_mining/profit.csv", "a") as f:
                writer = csv.writer(f)
                writer.writerows(x)
        except:
            writingerror.append(ticker)
            #print("Error with writing {}".format(ticker))
    except:
        readingerror.append(ticker)
        #print("Error with fetching {}".format(ticker))
# Print out cashtags that have no corresponding stock data
print("writignerrors: {}".format(writingerror))
print("readingerrors: {}".format(readingerror))

ODT
[[0.0, 'ODT']]
ESQ
[[-0.0409, 'ESQ']]
IQ
[[-0.1456, 'IQ']]
MOGU
[[0.1667, 'MOGU']]
UXIN
[[-0.0702, 'UXIN']]
CLBR
ALGRU
[[-0.001, 'ALGRU']]
SLDB
[[-0.1129, 'SLDB']]
MRNA
[[-0.1545, 'MRNA']]
HLG
[[0.2, 'HLG']]
SNDX
[[-0.0083, 'SNDX']]
VTEC
PARK
AGMH
[[0.1455, 'AGMH']]
BXG
[[0.0359, 'BXG']]
JG
[[-0.0222, 'JG']]
AAAU
[[-0.0084, 'AAAU']]
VPRL
BWB
[[-0.0032, 'BWB']]
UPWK
[[-0.0791, 'UPWK']]
ACAMU
[[-0.002, 'ACAMU']]
BILI
[[0.1469, 'BILI']]
VRCA
[[-0.125, 'VRCA']]
JFIN
[[0.4682, 'JFIN']]
ESTC
[[0.0, 'ESTC']]
ARLO
[[0.1946, 'ARLO']]
HZ
[[0.0, 'HZ']]
BAND
[[0.0062, 'BAND']]
CSSE
[[-0.2885, 'CSSE']]
CANG
[[0.0016, 'CANG']]
HHR
[[0.0744, 'HHR']]
DOMO
[[0.1471, 'DOMO']]
ARA
[[0.0, 'ARA']]
ORTX
[[-0.1765, 'ORTX']]
GTHX
[[0.0, 'GTHX']]
TC
[[0.0, 'TC']]
ADT
[[-0.0204, 'ADT']]
SAIL
[[-0.1275, 'SAIL']]
ZS
[[0.2, 'ZS']]
GNTY
[[0.0033, 'GNTY']]
IPSC
RWGEU
MMDMU
[[0.002, 'MMDMU']]
SBPH
[[-0.016, 'SBPH']]
MCB
[[-0.0447, 'MCB']]
XTF
OBNK
[[0.0, 'OBNK']]
YOGA
[[-0.1182, 'YOGA']]
GOSS
[[-0.0558, 'GOSS']]


ETTX
[[-0.2074, 'ETTX']]
ASNS
[[0.0792, 'ASNS']]
LBRT
[[0.0, 'LBRT']]
QTRX
[[0.0751, 'QTRX']]
WHD
[[-0.0403, 'WHD']]
GMR
CTK
[[-0.1791, 'CTK']]
HHHHU
[[0.0, 'HHHHU']]
CBUS
MTECU
[[0.004, 'MTECU']]
ESTA
[[-0.0481, 'ESTA']]
FTCH
[[0.0537, 'FTCH']]
QTT
[[0.7549, 'QTT']]
ELAN
[[0.1163, 'ELAN']]
AQ
[[0.0215, 'AQ']]
DDMXU
[[-0.001, 'DDMXU']]
LQDA
[[-0.1084, 'LQDA']]
BJ
[[-0.016, 'BJ']]
ALRN
[[0.0, 'ALRN']]
ROAD
[[-0.0692, 'ROAD']]
SWCH
[[-0.0396, 'SWCH']]
TNTR
[[0.0211, 'TNTR']]
ZYME
[[-0.037, 'ZYME']]
SIBN
[[-0.0448, 'SIBN']]
SPOT
[[-0.1018, 'SPOT']]
CIFS
[[-0.0538, 'CIFS']]
GPAQU
[[0.0, 'GPAQU']]
EVER
[[0.0192, 'EVER']]
ABLX
[[0.0, 'ABLX']]
KIDS
[[-0.1074, 'KIDS']]
SECO
[[-0.1736, 'SECO']]
PSIT
STNE
[[-0.0203, 'STNE']]
KZR
[[-0.1125, 'KZR']]
NCNA
[[0.0419, 'NCNA']]
IPIC
[[-0.1094, 'IPIC']]
PDD
[[0.0075, 'PDD']]
CSTX
GRAFU
AFGL
CODX
[[-0.0317, 'CODX']]
BJS
[[-0.0061, 'BJS']]
CELC
[[0.0, 'CELC']]
GRTS
[[-0.0632, 'GRTS']]
SENS
[[0.036, 'SENS']]
ALNA
[[-0.1683, 'ALNA']]
HVBH
ZLAB
[[0.1518, 'ZL

In [ ]:
#An example of the stock prices, this might be useful to look at.
pd.read_csv("/home/mma/Downloads/IPO_project/stock_mining/SURF.csv")

In [ ]:
df_profit = pd.read_csv("/home/mma/Downloads/IPO_project/stock_mining/profit.csv") 

# <span style = 'color : darkred'>  One observation per tweet </span style>

### <span style = 'color:darkgreen'> Keep one observation per tweet </span style>

The mechanism of tweets and retweets is the following:

- An original tweet is sent out
- Somebody retweets:
    - The retweet is preceded by **RT @username_original_tweet: [text of original tweet]**
    - The retweet count is incremented in the original tweet BUT in the retweet as well
 
That means, even if we don't have the original tweet, we can obtain in by removing the pattern **RT @username_original_tweet:** from the retweet. This is important because, as shown below, we won't always have access to the original tweet.

Another important step to take, is to delete all duplicates. Ultimately, we're going to be training and testing models on these tweets. Predicting a number of retweets using multiple training examples (i.e. retweets) per tweet would be erroneous. Hence we're going to take a few steps to try and avoid having  duplicate tweets in our dataset.



### <span style = 'color:darkgreen'> Rename & unnest important columns </span style>

In [ ]:
f_ipo = imp.reload(f_ipo)

In [ ]:
ls

In [ ]:
df = df.select("*").filter('lang == "en"')

In [ ]:
#Tweets can contain a lot of useless data or data that's hard to analyze. This includes the retweet patern,
#The hashtags, truncations and more.
def clean_text(text):
    # Remove pattern that signifies retweet
    clean_text = re.sub(pattern = "RT\s@[a-zA-Z0-9]+:\s", repl = "", string = text)
    clean_text = re.sub(r"\'s", " ", clean_text)
    clean_text = re.sub(r"\'ve", " have ", clean_text)
    clean_text = re.sub(r"can't", "cannot ", clean_text)
    clean_text = re.sub(r"n't", " not ", clean_text)
    clean_text = re.sub(r"i'm", "i am ", clean_text)
    clean_text = re.sub(r"\'re", " are ", clean_text)
    clean_text = re.sub(r"\'d", " would ", clean_text)
    clean_text = re.sub(r"\'ll", " will ", clean_text)
    clean_text = re.sub(pattern = "\$[A-Za-z]{1,5}", repl = " ticker ", string = clean_text)
    clean_text = re.sub(pattern =  "w/", repl = "with", string = clean_text)
    clean_text = re.sub(pattern = "http(s)?:\/\/[^\s]+", repl = "website", string = clean_text)
    clean_text = re.sub(pattern = "[\|!,\|?\.\[\]\"'(\n):\$\&\#\-\%\+]+", repl = " ", string = clean_text)
    clean_text = re.sub(pattern = ' +', repl = ' ', string = clean_text)
    clean_text = re.sub(pattern = "@[A-Za-z0-9]+", repl = "user", string= clean_text)
    clean_text = clean_text.lower()
    return clean_text

clean_text("RT @VIZIO: @blabla Ever wonder what it's like  new vizio 56465% +22 d24d1 24 class led smart tv 1080p new 2016 model 13899end date wednesday jun... website to " +            "work w/ at\n\n VIZIO? Check out our company profile: https://t.co/IkJXNZ9pL4 https://t.co/TWSixQQnU")
#We define a user defined function from the clean text function.   
udf_clean_text = udf(clean_text, StringType())   

In [ ]:
#function to check wheter a tweet contains a url.
def contains_url(isNotNull):
    if isNotNull == True:
        return 1
    else:
        return 0

In [ ]:
#Create a temorary view as DF_SQL
df.createOrReplaceTempView("DF_SQL")
#Filter all tweets to only contain the distinct tweets now
df_distinct_tweets = spark.sql("SELECT entities, extended_entities, favorite_count, id_str, in_reply_to_status_id_str,                       is_quote_status, lang, place, a.text, user, a.timestamp, a.retweet_count,                       a.cashtags, a.contains_ipo, a.contains_cashtag, a.contains_cashtag_with_spaces                       FROM DF_SQL a Inner JOIN                                    (SELECT text, retweet_count, min(timestamp) as timestamp                                    FROM DF_SQL                                    GROUP BY text, retweet_count                                    ) b                      ON  a.text = b.text                      AND a.retweet_count=b.retweet_count AND a.timestamp = b.timestamp").distinct()

#There's some columns with sub colums, here we take all the necesary data out of these columns,
#and when we have all the data drop the old column.
#the output gives us a scheme so we can understand the tweets better.
df_distinct_tweets = (df_distinct_tweets
        .withColumn("cashtag_count", udf(lambda x: len(x), IntegerType())(col("cashtags")))
    # ENTITIES  
        .withColumn("hashtags", col("entities.hashtags"))
        .withColumn("hashtag_count", udf(lambda x: len(x), IntegerType())(col("hashtags")))
        .withColumn("contains_hashtags", udf(f_ipo.contains_hashtags, IntegerType())(col("hashtags")))
        .withColumn("contains_url", udf(contains_url, IntegerType())(col("entities.media.display_url").isNotNull()))
        .withColumn("user_mentions_count", udf(lambda x: len(x), IntegerType())(col("entities.user_mentions")))
        .drop("entities")
    # USER
        .withColumn("user_seniority", udf(f_ipo.user_seniority, DoubleType())("user.created_at"))
        .withColumn("user_description", col("user.description"))
        .withColumn("user_favourites_count", col("user.favourites_count"))
        .withColumn("user_friends_count", col("user.friends_count"))
        .withColumn("user_followers_count", col("user.followers_count"))
        #.withColumn("user_description_clean", udf_clean_text(col("user_description")))
        .drop("user")
    # PLACE
        .drop("place")
    # EXTENDED_ENTITIES
        .drop("extended_entities")
        .withColumn("retweet_count_double", col("retweet_count").cast("double"))
        .withColumn("day_of_week", udf_day_of_week(col("timestamp")))
        .withColumn("text_clean", udf_clean_text(col("text")))
        .withColumn("clean_text_len", udf(lambda x: len(x), IntegerType())("text_clean"))
        .withColumn("nbr_words",  udf(lambda x: len(x.split(" ")), IntegerType())(col("text_clean")))
                     )

df_distinct_tweets.printSchema()



In [ ]:
#saving distinct tweets as parquet file for further use in Machine Learning
df_distinct_tweets.write.mode('overwrite').parquet("/home/mma/Downloads/IPO_Mining/disctinct_tweets_test/ML_features22.parquet")

In [ ]:
#Reading in the above parquet file
df_distinct_tweets = spark.read.parquet("/home/mma/Downloads/IPO_Mining/disctinct_tweets_test/ML_features22.parquet")

In [ ]:
#We clean our tweets with the function we created before. Now they'll contain less useless info
text_df = df_distinct_tweets.select("text").toPandas() # converting pyspark DF to pandas DF
text_df["clean"] = text_df["text"].apply(lambda x: clean_text(x))  # applying clean_text function
text_df.sample(100)

In [ ]:

def RF_classification(model_description, cat_columns, num_columns, output_column = "retweet_binary"):
    stringIndexers = []
    indexed_cols = []

    encoders = []
    encoded_cols = []
    #add a numbered index to all categories from the columns
    #this means all rows with category "a" will be "1", "b" will be "2", and so forth (as an example)
    for column in cat_columns:
        inCol= column
        outCol = column + "indexed"
        stringIndexer = StringIndexer(inputCol = inCol, outputCol=outCol)

        stringIndexers.append(stringIndexer)
        indexed_cols.append(outCol)

    for column in indexed_cols:
        inCol= column
        outCol = column + "encoded"
        encoder = OneHotEncoder(inputCol=inCol, outputCol=outCol)

        encoders.append(encoder)
        encoded_cols.append(outCol)

    binarizer = Binarizer(inputCol = "retweet_count_double", outputCol = output_column, threshold=0.9)

    assembler = VectorAssembler(inputCols= num_columns + encoded_cols, outputCol="features")
    
    pipeline = Pipeline(stages= stringIndexers + encoders + [assembler] + [binarizer])
    pipelineModel = pipeline.fit(df_distinct_tweets)

    transformed_df = (pipelineModel.transform(df_distinct_tweets)
                                   .select("features", 
                                            col("retweet_binary").cast("integer"), 
                                            "retweet_count")
                     )
    
    transformed_df = transformed_df
    
    train, validation, test = transformed_df.randomSplit([0.8, 0.1, 0.1], seed = 42)
    
    rf = RandomForestClassifier(numTrees=500, labelCol="retweet_binary")
    rf_fitted = rf.fit(train)

    train_predictions = rf_fitted.transform(train)
    validation_predictions = rf_fitted.transform(validation)
    
    AUC_evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="retweet_binary", metricName="areaUnderROC")

    AUC_train = AUC_evaluator.evaluate(train_predictions)
    AUC_validation = AUC_evaluator.evaluate(validation_predictions)
    print("###################" + model_description.upper() + "##################")
    print("TRAIN AUC:       {}".format(AUC_train))
    print("TRAIN VALIDATION:{}".format(AUC_validation))

** time_of_day? **


In [ ]:
all_columns = df_distinct_tweets.columns
print(all_columns)

non_text_num_cols = ["user_seniority", "user_favourites_count", "user_friends_count", "user_followers_count"]
non_text_cat_cols = ["day_of_week"]
text_num_cols = ["cashtag_count", "hashtag_count"]
text_cat_cols = ["lang", "contains_cashtag", "contains_cashtag_with_spaces", "contains_ipo", "contains_hashtags", "contains_url"]
dependent_variable = ["retweet_count"]
text = ["clean_text"]



In [ ]:
# In all the notebooks there are problems with the random forest code, i am not sure if it is the code or the Data set used that triggers these problems.
get_ipython().run_cell_magic('time', '', 'RF_classification(model_description = "All inclusive model", \n                        cat_columns = text_cat_cols + non_text_cat_cols, \n                      num_columns = text_num_cols + non_text_num_cols)')


In [ ]:
# also random forest error
get_ipython().run_cell_magic('time', '', 'RF_classification(model_description = "Text only model", \n                  cat_columns = text_cat_cols, \n                  num_columns = text_num_cols)')


# <span style = 'color:darkred'> Deep learning model using keras </span style>


In [ ]:
#This is the GoogleNews-Vectors, this is a .bin file that also has not been provided by the creator so we downloaded one from the internet.
#This still gives us an error because this propably is not the same version useds by the creator. origin error => unkown



#get_ipython().run_cell_magic('time', '', 'from gensim.models import KeyedVectors\n\nword2vec = KeyedVectors.load_word2vec_format("/home/sachadubrulle/Data/W2V/GoogleNews-vectors-negative300.bin", binary=True)\n\nprint(\'Found %s word vectors of word2vec\' % len(word2vec.vocab))\n\nEMBEDDING_DIM = 300')
get_ipython().run_cell_magic('time', '', 'from gensim.models import KeyedVectors\n\nword2vec = KeyedVectors.load_word2vec_format("googlevector/GoogleNews-vectors-negative300.bin", binary=True)\n\nprint(\'Found %s word vectors of word2vec\' % len(word2vec.vocab))\n\nEMBEDDING_DIM = 300')


In [ ]:
#Importing packages for text mining
from gensim.models import Word2Vec, KeyedVectors
KeyedVectors.load_word2vec_format("googlevector/GoogleNews-vectors-negative300.bin", binary=True, limit=500)

In [ ]:
# converting into Panda DF to export
df_pandas = df_distinct_tweets.toPandas()

In [ ]:
df_pandas

In [ ]:
# Exporting the file 
df_pandas.to_csv("/home/mma/Downloads/IPO_Mining/df_tweets/tweets.csv", header=True, index=False, encoding='utf-8')



In [ ]:
'''# This never worked in the first place
#train_labels = df_train(pd.read_csv("df_tweets/tweets.csv"))
#df_train['cashtags'].describe()
df_new = pd.read_csv(open('/home/mma/Downloads/IPO_Mining/df_tweets/tweets.csv','rU'), encoding='utf-8', engine='c')
#df_new = pd.read_csv('/home/mma/Downloads/IPO_Mining/df_tweets/tweets.csv', encoding='utf8')
#df_train['cashtags'].describe()
#train_labels = df_train["row name"]

def data_prep_LSTM(df, dataset =  ""):
    # shorten sequence to maximum length
   
    df["Q1_max_len"] = df["Q1_clean"].apply(lambda x: " ".join(x.split(" ")[0:MAX_SEQ_LENGTH]))
    df["Q2_max_len"] = df["Q2_clean"].apply(lambda x: " ".join(x.split(" ")[0:MAX_SEQ_LENGTH]))
    #df["num_words"] = df["Q1_max_len"].apply(lambda x: len(x.split(" ")))
    
    #df.drop(["Q1_clean", "Q2_clean"], axis = 1, inplace = True)
    #df.drop(["clean_text"], axis = 1, inplace = True)
    
    if dataset == "train":
            df.drop(["is_duplicate"], axis = 1, inplace = True) 
    return df


tweetcount = df_new['id_str'].count() //4 * 3
df_train = df_new[:tweetcount]
df_test = df_new[tweetcount:]
df_train_done = data_prep_LSTM(df_train, "train")
df_test_done = data_prep_LSTM(df_test)
    '''

In [ ]:
#summary of the text
df_distinct_tweets.select("nbr_words").describe().show()

# <span style = 'color:darkred'> Basic retweet model Spark </span style>

When a tweet is initially fetched, the retweet_count is almost inevitably equal to zero as the instance is fetched immediately after it appears on the social network. Hence to obtain the number of retweets to train a retweet prediction model, we must fetch the tweets again at a later point in time.

Few problems arising with this:

- A considerable number of tweets have been deleted (either only the tweet, or even the account of the sender)
- With regards to storing the tweet object:
    - If it is done at the initial fetch: 
        - the storage space needed is considerable
        - we can filter out tweets that we aren't interested in based to avoid fetching irrelevant tweets a second time
    - If we only save the tweet id in the initial fetch:
        - Less storage space is needed initially
        - We cannot filter based on the text in the tweets and hence we will have to refetch all tweets, which might be a time costly, as twitter has some pretty low rate limits
    - Solution could be to:
        - save only the tweet id and the text at initial fetch, which can then be used to filter without requiring too much storage
        - Save the full object when tweet is fetched a second time
    
            




### <span style = 'color:darkgreen'> Cashtag over time </span style>

In [ ]:
df = df_distinct_tweets

In [ ]:
#Function to plot the Tickers
def plot_ticker(df, plot_ticker, image_save_dir):
    
    #print "Enter your stock as a string, i.e. with quotation marks"
    filtered_CT_per_TS = df.select("timestamp", explode("cashtags").alias("cashtag")).filter("cashtag == '{}'".format(plot_ticker))
    #print(filtered_CT_per_TS)
    # cashtag per timestamp
    CT_TS_pd = filtered_CT_per_TS.toPandas()
    CT_TS_pd["date"] = CT_TS_pd["timestamp"].apply(lambda x: x.date())
    CT_TS_pd["date_str"] = CT_TS_pd["date"].apply(lambda x: str(x))
    CT_TS_pd.drop("timestamp", axis = 1, inplace = True)
    #print(CT_TS_pd)
    
    #cashtag count per day
    CT_TWTCOUNT_DATE = pd.DataFrame(CT_TS_pd["date_str"].value_counts()).sort_index(ascending = True).reset_index(drop = False)
    CT_TWTCOUNT_DATE.columns = ["date", "count"]
    #print(CT_TWTCOUNT_DATE)
    # stock prices
    stock_prices_ticker = pd.read_csv("/home/mma/Downloads/IPO_Mining/stock_mining/{}.csv".format(plot_ticker))
    #[["Date", "Close"]].sort_values("Date", ascending = True)
    stock_prices_ticker = stock_prices_ticker.drop(['open','high','low','volume'],axis=1)
    stock_prices_ticker = stock_prices_ticker.rename(columns= {'b\'timestamp':'date'})
    stock_prices_ticker = stock_prices_ticker.sort_values("date", ascending = True)
    #print(stock_prices_ticker)
    full_stock_df = pd.merge(left = stock_prices_ticker, right = CT_TWTCOUNT_DATE, on = "date", how = "right").sort_values("date", ascending = True)
    print (full_stock_df)
    full_stock_df["close"].fillna(value = 0, inplace = True)
    full_stock_df.reset_index(drop = True, inplace = True)
    #print (full_stock_df)
    
    # PLOTTING
    fig = plt.figure(figsize = (20,7))

    ax1 = fig.add_subplot(111)
    ax1.set_title(plot_ticker)
    ax1.plot(full_stock_df["count"], color= colors["orange"], linewidth = 2.5)
    ax1.set_ylabel('count')
    
    ax2 = ax1.twinx()
    ax2.plot(full_stock_df["close"].loc[(full_stock_df.close != 0)], color = colors["blue"], linewidth = 2.5)
    ax2.set_ylabel('close')
    
    ax1.xaxis.set_major_locator(YearLocator())
    ax1.xaxis.set_major_formatter(DateFormatter('%Y'))
    ax1.xaxis.set_minor_locator(MonthLocator())
    # format the coords message box
    ax2.format_xdata = DateFormatter('%Y-%m-%d')
    ax2.grid(True)
    
    print("Count is orange, stock price is blue")
    #fig.savefig('{}{}.png'.format(image_save_dir, plot_ticker))
    #fig.show()
    #return full_stock_df

In [ ]:
df.select("cashtags").show(200)

In [ ]:
#Filtering required cashtags
#filtered_CT_per_TS = 
df.where(col("cashtags").isin({"SPOT"}))

#df.filter(col("cashtags").isin('SPOT')).show()

In [ ]:
#converting into pands and saving in the object
pandas_df = filtered_CT_per_TS.toPandas()


In [ ]:
# plotting the Tickers count of tweets vs  price
plot_ticker(df, "AMZN", "ipoFiles/")

In [ ]:
# Most plot_tickers have problems with the : Cannot convert tz-naive Timestamp, use tz_localize to localize = > problem with the timestamp convertion to panda's.
# look at the code of Twitter filter or twitter analyse for a solution, timestamps have also been used. 
# yet the plots do not give this error. the conversion has been donde differently
# Here however it is propably because my limited data set did not haave a ticker called "SNAP" so normaly this would also give the Timestamp error. this df is just  missing the ticker
plot_ticker(df, "SPOT", "ipoFiles/")

# Plot tickers

In [2]:
def plot_ticker_data_prep(df, plot_ticker):
    
    #print "Enter your stock as a string, i.e. with quotation marks"
    filtered_CT_per_TS = df.select("timestamp", explode("cashtags").alias("cashtag")).filter("cashtag == '{}'".format(plot_ticker))
    
    # cashtag per timestamp
    CT_TS_pd = filtered_CT_per_TS.toPandas()
    CT_TS_pd["date"] = CT_TS_pd["timestamp"].apply(lambda x: x.date())
    CT_TS_pd["date_str"] = CT_TS_pd["date"].apply(lambda x: str(x))
    CT_TS_pd.drop("timestamp", axis = 1, inplace = True)
    
    #cashtag count per day
    CT_TWTCOUNT_DATE = pd.DataFrame(CT_TS_pd["date_str"].value_counts()).sort_index(ascending = True).reset_index(drop = False)
    CT_TWTCOUNT_DATE.columns = ["date", "count"]
    
    # stock prices
    stock_prices_ticker = pd.read_csv("stock_mining/{}.csv".format(plot_ticker))
    #[["date", "close"]].sort_values("date", ascending = True)
    stock_prices_ticker = stock_prices_ticker.drop(['open','high','low','volume'],axis=1)
    print(stock_prices_ticker)
    stock_prices_ticker = stock_prices_ticker.rename(columns= {'timestamp':'date','close':'closing_price'})
    #print(stock_prices_ticker)
    stock_prices_ticker = stock_prices_ticker.sort_values('date', ascending = True)
    #stock_prices_ticker.columns = ["date", "closing_price"]
    
    full_stock_df = pd.merge(left = stock_prices_ticker, right = CT_TWTCOUNT_DATE, on = "date", how = "right").sort_values("date", ascending = True)
    full_stock_df["closing_price"].fillna(value = 0, inplace = True)
    full_stock_df.reset_index(drop = True, inplace = True)
    return full_stock_df
    

In [3]:
# again "SNAP"ticker is propably just missing.
stock = "SPOT"

stock_df = plot_ticker_data_prep(df, stock)
print(stock_df)

NameError: name 'df' is not defined

In [ ]:
# because previous cell gives an error the stock_df is never created so this ceel can not run
stock_df["timestamp"] = stock_df["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
print(stock_df)

In [ ]:
# because previous cell gives an error the stock_df is never created so this ceel can not run

# FOR SNAPCHAT
stock_df["timestamp"] = stock_df["timestamp"].apply(lambda x: x.year)
#stock_df = stock_df.ix[stock_df["timestamp"].apply(lambda x: x.year) == 2017]

In [ ]:
'''stock_df = stock_df.ix[stock_df["timestamp"] == 2018]
print(stock_df)'''

In [ ]:
# because previous cell gives an error the stock_df is never created so this ceel can not run

'''plt.rc('font', size=15)
plt.rc('axes', titlesize=20)

fig = plt.figure(figsize = (20,10))


ax1 = fig.add_subplot(111)
ax1.set_title('stock')

ax1.plot(stock_df["date"], stock_df["count"], 
         color= colors["orange"], 
         linewidth = 2.5, 
         label = "Tweet count")

ax1.grid(True, axis = 'x')
ax1.format_xdata = DateFormatter('%Y-%m-%d')
ax1.xaxis.set_minor_locator(MonthLocator())
ax1.set_ylabel('Tweet count')


ax2 = ax1.twinx()
ax2.plot(stock_df["date"].ix[(stock_df.closing_price != 0)], 
         stock_df["closing_price"].ix[(stock_df.closing_price != 0)], 
         color = colors["blue"], 
         linewidth = 2.5,
         label = "Closing price")
ax2.set_ylabel('Closing price')

legend = ax1.legend(loc="upper left", shadow=True)
legend = ax2.legend(loc="upper right", shadow=True)


fig.tight_layout()

fig.savefig('{}.png'.format(stock))

'''

In [ ]:
#This is just to see how long this notebook ran
datetime.now() - date_start